In [6]:
!pip3 install kaggle

In [13]:
!kaggle competitions download -c toxic-comments-classification-apdl-2021
!unzip toxic-comments-classification-apdl-2021.zip -d data

toxic-comments-classification-apdl-2021.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  toxic-comments-classification-apdl-2021.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_data.csv      
  inflating: data/train_data.csv     


In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [24]:
train = pd.read_csv('data/train_data.csv')
test = pd.read_csv('data/test_data.csv')

In [23]:
train.toxic.describe()

count    10809.000000
mean         0.333796
std          0.471590
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: toxic, dtype: float64

In [28]:
test.sample(5)

,comment_id,comment
961,961,"Избить, тело сжечь, а то что останется пустить..."
2015,2015,"Спасибо, а то магазин в телефоне пустовал.\n"
2119,2119,"Не обращал внимания, просто привел цитату из с..."
2721,2721,О чем речь сейчас? Использовать обычный поводо...
3127,3127,"Хуита что-то какая-то. Ни сисек, ни жопы, ряза..."
